In [6]:
!pip install opencv-python scikit-learn pandas numpy matplotlib tensorflow-addons
!pip install tensorflow==2.12 keras==2.12
!pip install transformers datasets torchvision

In [2]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121  # Changed to DenseNet121 for speed
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, Multiply, GlobalAveragePooling2D, Dense, Input, Softmax, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import kagglehub

2025-07-02 19:48:44.808516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751485725.042181      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751485725.109336      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import cv2
import numpy as np

def preprocess_image(image_path):
    # Read as grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    
    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)

    # Resize using bicubic interpolation
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)

    # Gaussian Blur
    img = cv2.GaussianBlur(img, (5, 5), 0)

    # Thresholding
    _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # Erosion
    kernel = np.ones((3, 3), np.uint8)
    img = cv2.erode(img, kernel, iterations=1)

    # Dilation
    img = cv2.dilate(img, kernel, iterations=1)

    # Normalize and convert to 3-channel
    img = img.astype(np.float32) / 255.0
    img = np.stack((img,) * 3, axis=-1)
    
    return img

In [5]:
from transformers import ViTImageProcessor, ViTForImageClassification
import torch
from datasets import load_dataset
from PIL import Image
import requests
from sklearn.metrics import classification_report
import torch
from torch.optim import AdamW 
from transformers import ViTFeatureExtractor
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [6]:
path = kagglehub.dataset_download("adityamahimkar/iqothnccd-lung-cancer-dataset")
print("Path to dataset files:", path)
path = kagglehub.dataset_download("justinkirby/the-cancer-imaging-archive-lidcidri")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/iqothnccd-lung-cancer-dataset
Path to dataset files: /kaggle/input/the-cancer-imaging-archive-lidcidri


In [7]:
data_dir = "/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset"


In [8]:
from tensorflow.keras.utils import to_categorical

def load_dataset_with_preprocessing(data_dir, labels_map):
    images = []
    labels = []
    
    for label_name, label_idx in labels_map.items():
        label_folder = os.path.join(data_dir, label_name)
        if not os.path.exists(label_folder):
            print(f"[!] Skipping missing folder: {label_folder}")
            continue

        for filename in os.listdir(label_folder):
            if filename.startswith('.'):  # Skip hidden files like .DS_Store
                continue

            img_path = os.path.join(label_folder, filename)
            img = preprocess_image(img_path)

            if img is None:
                print(f"[!] Failed to load image: {img_path}")
                continue

            images.append(img)
            labels.append(label_idx)

    print(f"[✓] Loaded {len(images)} images from {data_dir}")
    X = np.array(images)
    y = to_categorical(np.array(labels), num_classes=len(labels_map))
    return X, y

# Example labels_map (adjust if different)
labels_map = {'Bengin cases': 0, 'Malignant cases': 1, 'Normal cases': 2}

from sklearn.model_selection import train_test_split

X, y = load_dataset_with_preprocessing(data_dir, labels_map)
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Train data shape: {trainX.shape}, Train labels shape: {trainY.shape}")
print(f"Test data shape: {testX.shape}, Test labels shape: {testY.shape}")

[✓] Loaded 1097 images from /kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset
Train data shape: (877, 224, 224, 3), Train labels shape: (877, 3)
Test data shape: (220, 224, 224, 3), Test labels shape: (220, 3)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: benign, malignant, normal
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("TrainX shape:", trainX.shape)
print("TrainY shape:", trainY.shape)
print("TestX shape:", testX.shape)
print("TestY shape:", testY.shape)

# ✅ Train the model on preprocessed data
model.fit(
    trainX, trainY,
    validation_data=(testX, testY),
    epochs=10,
    batch_size=32
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1751485809.431718      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751485809.432359      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


TrainX shape: (877, 224, 224, 3)
TrainY shape: (877, 3)
TestX shape: (220, 224, 224, 3)
TestY shape: (220, 3)
Epoch 1/10


I0000 00:00:1751485813.990612      85 service.cc:148] XLA service 0x7d6e3000ad80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751485813.991504      85 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751485813.991528      85 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751485814.285615      85 cuda_dnn.cc:529] Loaded cuDNN version 90300


 5/28 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3645 - loss: 9.8109

I0000 00:00:1751485819.042894      85 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


28/28 ━━━━━━━━━━━━━━━━━━━━ 13s 228ms/step - accuracy: 0.5317 - loss: 5.6198 - val_accuracy: 0.8545 - val_loss: 0.4336
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.8889 - loss: 0.3061 - val_accuracy: 0.9636 - val_loss: 0.1090
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9796 - loss: 0.0697 - val_accuracy: 0.9864 - val_loss: 0.0837
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9828 - loss: 0.0506 - val_accuracy: 0.9773 - val_loss: 0.0477
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9933 - loss: 0.0300 - val_accuracy: 0.9773 - val_loss: 0.0572
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9943 - loss: 0.0232 - val_accuracy: 0.9864 - val_loss: 0.0682
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9987 - loss: 0.0134 - val_accuracy: 0.9864 - val_loss: 0.0525
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9969 - loss: 0.0109 - val_accuracy: 0.9864 - val_loss: 

In [10]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to ViT's input size
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

from torch.utils.data import random_split

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [11]:
from transformers import ViTForImageClassification

num_classes = len(train_loader.dataset.dataset.classes)  # e.g., 3 for benign, malignant, normal

model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=num_classes
)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import copy

best_val_loss = float('inf')
best_model_state = None


In [13]:
import torch
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

best_val_loss = float('inf')
best_model_state = None

for epoch in range(15):
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs).logits
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).logits
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    accuracy = correct / total

    print(f"Epoch {epoch+1}, Train Loss: {total_loss:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")

    # =========================
    # Save best model callback
    # =========================
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = copy.deepcopy(model.state_dict())
        print("✅ Best model updated.")

# Save the best model
torch.save(best_model_state, "best_vit_model.pth")
print("🎉 Best model saved to 'best_vit_model.pth'")


Epoch 1, Train Loss: 18.2770
Validation Loss: 0.3821, Accuracy: 0.8864
✅ Best model updated.
Epoch 2, Train Loss: 10.7182
Validation Loss: 0.2781, Accuracy: 0.9045
✅ Best model updated.
Epoch 3, Train Loss: 8.3338
Validation Loss: 0.2512, Accuracy: 0.9091
✅ Best model updated.
Epoch 4, Train Loss: 7.8620
Validation Loss: 0.3193, Accuracy: 0.8682
Epoch 5, Train Loss: 8.1677
Validation Loss: 0.2546, Accuracy: 0.8909
Epoch 6, Train Loss: 7.0631
Validation Loss: 0.2825, Accuracy: 0.8455
Epoch 7, Train Loss: 6.0889
Validation Loss: 0.2757, Accuracy: 0.8273
Epoch 8, Train Loss: 5.6145
Validation Loss: 0.1694, Accuracy: 0.9500
✅ Best model updated.
Epoch 9, Train Loss: 4.4952
Validation Loss: 0.1332, Accuracy: 0.9500
✅ Best model updated.
Epoch 10, Train Loss: 3.2419
Validation Loss: 0.1402, Accuracy: 0.9545
Epoch 11, Train Loss: 2.1025
Validation Loss: 0.1193, Accuracy: 0.9682
✅ Best model updated.
Epoch 12, Train Loss: 3.1593
Validation Loss: 0.0895, Accuracy: 0.9864
✅ Best model updated.
E

In [14]:
from sklearn.metrics import classification_report

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        logits = model(inputs).logits
        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=train_loader.dataset.dataset.classes))


                 precision    recall  f1-score   support

   Bengin cases       0.58      0.90      0.71        20
Malignant cases       0.99      1.00      1.00       116
   Normal cases       0.97      0.83      0.90        84

       accuracy                           0.93       220
      macro avg       0.85      0.91      0.87       220
   weighted avg       0.95      0.93      0.93       220



In [15]:
torch.save(model.state_dict(), "vit_lung_cancer_model.pth")